<a href="https://colab.research.google.com/github/javohir-ergashboyev/NLP/blob/main/Fake_Real_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import spacy

In [2]:
df=pd.read_csv('Fake_Real_Data.csv')
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [3]:
df.shape

(9900, 2)

In [5]:
df['label'].value_counts()

Fake    5000
Real    4900
Name: label, dtype: int64

In [7]:
df['label_num']=df['label'].map({'Fake':1, 'Real':0})

In [8]:
df.sample(5)

,Text,label,label_num
4892,Senate Republicans struggle to salvage healthc...,Real,0
6011,Republican Tiberi to leave Congress in January...,Real,0
8333,"U.S. retaliates against Russia, orders closure...",Real,0
7938,Trump nominates former fund manager Quintenz t...,Real,0
5464,MSNBC Host Has PERFECT Response To Coughing G...,Fake,1


In [12]:
!python -m spacy download en_core_web_lg

2024-01-28 14:14:06.857123: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 14:14:06.857180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 14:14:06.858649: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 14:14:08.162114: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [13]:
nlp=spacy.load('en_core_web_lg')
doc=nlp('Senate Republicans struggle to salvage')
doc.vector.shape

(300,)

In [14]:
df['text_vec']=df['Text'].apply(lambda x: nlp(x).vector)

In [15]:
df.sample(5)

,Text,label,label_num,text_vec
1310,WATCH: Republican Kentucky Governor Calls For...,Fake,1,"[-1.4377588, 1.6234474, -3.0025935, -0.1930364..."
899,Congressman will not seek relection amid misco...,Real,0,"[-1.9090476, 0.49905652, -1.6243116, 0.506832,..."
5207,"Trump OKs sending 4,000 more troops to Afghani...",Real,0,"[-1.1831759, -0.5564311, -1.5837839, 2.0103786..."
6585,WATCH: Tucker Carlson Just Heaped Praise On T...,Fake,1,"[-1.3334175, 0.3879476, -2.0422006, -0.0786396..."
6438,Trump to nominate NFL team owner Johnson as U....,Real,0,"[-1.4318334, 1.1315329, -1.6039706, 0.7074922,..."


In [16]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(df['text_vec'], df['label_num'],
                                           test_size=0.2, random_state=100,
                                           stratify=df['label_num'])

In [17]:
xtrain2=np.stack(xtrain)
xtest2=np.stack(xtest)

In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

In [23]:
scaler=MinMaxScaler()
scaled_xtrain=scaler.fit_transform(xtrain2)
scaled_xtest=scaler.transform(xtest2)
clf=MultinomialNB()

clf.fit(scaled_xtrain, ytrain)

MultinomialNB()

In [24]:
ypred=clf.predict(scaled_xtest)
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       980
           1       0.96      0.95      0.95      1000

    accuracy                           0.95      1980
   macro avg       0.95      0.95      0.95      1980
weighted avg       0.95      0.95      0.95      1980



In [27]:
from sklearn.neighbors import KNeighborsClassifier

cls=KNeighborsClassifier(n_neighbors=5, metric='euclidean')
cls.fit(scaled_xtrain, ytrain)

KNeighborsClassifier(metric='euclidean')

In [28]:
ypred=cls.predict(scaled_xtest)
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.99      0.98      0.99      1000

    accuracy                           0.99      1980
   macro avg       0.99      0.99      0.99      1980
weighted avg       0.99      0.99      0.99      1980

